In [2]:
import pandas as pd


data = pd.read_csv("/content/drive/My Drive/crawling/alba.csv",index_col=[0])

data

,title,area,pay,time,recently,link,lable
0,"2월28일/남양주 물류창고/출,퇴근 셔틀운행/실지급11만2",\n\n스크랩\n서울 노원구\n,일급,\r\n\t\t\t\t\t15:30~00:30\r\n\t\t\t\t\t,5분전,http://www.albamon.com//recruit/view/gi?AL_GI_...,True
1,2월28일 냉장 풀타임/실지급11만2천원/당일지급/하루가능,\n\n스크랩\n서울 송파구\n,일급,\r\n\t\t\t\t\t15:30~00:30\r\n\t\t\t\t\t,5분전,http://www.albamon.com//recruit/view/gi?AL_GI_...,True
2,[바로출근/익일지급가능/저녁출근]쿠팡 강서1캠프 장/단,\n\n스크랩\n서울 강서구\n,일급,\r\n\t\t\t\t\t18:30~22:00\r\n\t\t\t\t\t,6분전,http://www.albamon.com//recruit/view/gi?AL_GI_...,True
3,"소분, 포장 아르바이트",\n\n스크랩\n서울 영등포\n,시급,\r\n\t\t\t\t\t시간협의\r\n\t\t\t\t\t,8분전,http://www.albamon.com//recruit/view/gi?AL_GI_...,True
4,"블로그포스팅 재택알바 (단기알바,부업,투잡가능,사무알",\n\n스크랩\n서울 강서구\n,건별,\r\n\t\t\t\t\t시간협의\r\n\t\t\t\t\t,8분전,http://www.albamon.com//recruit/view/gi?AL_GI_...,True
...,...,...,...,...,...,...,...
95,농협중앙회 통합 it센터 출입자 열체크요원 모집,\n\n스크랩\n서울 전체 \n,시급,\r\n\t\t\t\t\t08:00~11:00\r\n\t\t\t\t\t,1시간전,http://www.albamon.com//recruit/view/gi?AL_GI_...,True
96,"빠른지급 [ 쉬운업무] 내일 바로가능, 피킹 , 다스업무",\n\n스크랩\n서울 성동구\n,일급,\r\n\t\t\t\t\t10:00~19:00\r\n\t\t\t\t\t,1시간전,http://www.albamon.com//recruit/view/gi?AL_GI_...,True
97,[단기/꿀알바] 게임사 IGS 게임테스트 QA사원 채용,\n\n스크랩\n서울 영등포\n,시급,\r\n\t\t\t\t\t10:00~19:00\r\n\t\t\t\t\t,1시간전,http://www.albamon.com//recruit/view/gi?AL_GI_...,True
98,"빠른지급 [ 쉬운업무] 내일 바로가능, 피킹 , 다스업무",\n\n스크랩\n서울 노원구\n,일급,\r\n\t\t\t\t\t10:00~19:00\r\n\t\t\t\t\t,1시간전,http://www.albamon.com//recruit/view/gi?AL_GI_...,True


In [3]:
!pip install soynlp

     |████████████████████████████████| 419kB 2.8MB/s 


In [4]:
!sudo apt-get install curl git
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.58.0-2ubuntu3.8).
git is already the newest version (1:2.17.1-1ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.
Installing automake (A dependency for mecab-ko)
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Hit:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Relea

In [5]:
!pip install gensim

In [6]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 252kB/s 
     |████████████████████████████████| 2.4MB 41.0MB/s 
     |████████████████████████████████| 92kB 12.8MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [7]:
from konlpy.tag import Mecab
from gensim.models import Word2Vec


mecab = Mecab()
print(mecab.morphs(data.title[0]))
print(mecab.nouns(data.title[0]))

['2', '월', '28', '일', '/', '남양주', '물류', '창고', '/', '출', ',', '퇴근', '셔틀', '운행', '/', '실지', '급', '11', '만', '2']
['월', '일', '남양주', '물류', '창고', '퇴근', '셔틀', '운행', '급', '만']


In [8]:
f = open("/content/drive/My Drive/crawling/koreanStopwords.txt", 'r')
lines = f.readlines()
stopwords = []
for line in lines:
    line = line.replace('\n', '')
    line = line.replace('\t', '')
    stopwords.append(line)
f.close()
stopwords

['이',
 '있',
 '하',
 '것',
 '들',
 '그',
 '되',
 '수',
 '이',
 '보',
 '않',
 '없',
 '나',
 '사람',
 '주',
 '아니',
 '등',
 '같',
 '우리',
 '때',
 '년',
 '가',
 '한',
 '지',
 '대하',
 '오',
 '말',
 '일',
 '그렇',
 '위하',
 '때문',
 '그것',
 '두',
 '말하',
 '알',
 '그러나',
 '받',
 '못하',
 '일',
 '그런',
 '또',
 '문제',
 '더',
 '사회',
 '많',
 '그리고',
 '좋',
 '크',
 '따르',
 '중',
 '나오',
 '가지',
 '씨',
 '시키',
 '만들',
 '지금',
 '생각하',
 '그러',
 '속',
 '하나',
 '집',
 '살',
 '모르',
 '적',
 '월',
 '데',
 '자신',
 '안',
 '어떤',
 '내',
 '경우',
 '명',
 '생각',
 '시간',
 '그녀',
 '다시',
 '이런',
 '앞',
 '보이',
 '번',
 '나',
 '다른',
 '어떻',
 '여자',
 '개',
 '들',
 '사실',
 '이렇',
 '점',
 '싶',
 '말',
 '정도',
 '좀',
 '원',
 '잘',
 '통하',
 '소리',
 '놓']

In [9]:
data2 = data.copy()
stop_word = ["#", "[", "]", "/", ",", "+", "-", "(", ")", "_", "%"] + stopwords
vocab = {}

for i, t in enumerate(data2.title):
    word_token = mecab.morphs(t)
    temp = []
    for w in word_token:
        if w not in stop_word:
            temp.append(w)
            if w not in vocab:
                vocab[w] = 0
            vocab[w] += 1
    data2.title[i] = temp

vocab_sorted = sorted(vocab.items(), key=(lambda x:x[1]), reverse=True)
print(vocab_sorted)
data2.title[0]

[('쿠팡', 50), ('주간', 44), ('물류', 37), ('가능', 35), ('야간', 32), ('단기', 31), ('지급', 23), ('원단', 23), ('서울역', 23), ('출발', 23), ('추가', 23), ('인센티브', 23), ('동반', 19), ('팀', 19), ('알바', 17), ('피', 17), ('업무', 17), ('바', 16), ('킹', 16), ('직영', 15), ('2', 14), ('당일', 14), ('급구', 14), ('C', 14), ('바로', 12), ('서울', 12), ('내일', 11), ('만', 10), ('HUB', 10), ('상차', 10), ('모집', 10), ('장지', 9), ('하루', 8), ('꿀', 8), ('광주', 8), ('조', 8), ('마켓', 8), ('컬리', 8), ('112', 8), ('000', 8), ('센터', 8), ('초보', 7), ('쉬운', 7), ('다스', 7), ('28', 6), ('동탄', 6), ('분류', 6), ('시', 6), ('패킹', 6), ('퇴근', 5), ('셔틀', 5), ('실지', 5), ('급', 5), ('11', 5), ('출근', 5), ('포장', 5), ('3', 5), ('근무', 5), ('홈', 5), ('쇼핑', 5), ('택배', 5), ('상', 5), ('하차', 5), ('인원', 5), ('최대', 5), ('15', 5), ('익일', 4), ('아르바이트', 4), ('주휴', 4), ('주말', 4), ('합니다', 4), ('E', 4), ('29', 4), ('최고', 4), ('빠른', 4), ('냉장', 3), ('풀타임', 3), ('천', 3), ('저녁', 3), ('1', 3), ('친구', 3), ('덕평', 3), ('일당', 3), ('수당', 3), ('이동', 3), ('오후', 3), ('단순', 3), ('단가', 3), ('게', 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


['2',
 '28',
 '남양주',
 '물류',
 '창고',
 '출',
 '퇴근',
 '셔틀',
 '운행',
 '실지',
 '급',
 '11',
 '만',
 '2']

In [10]:
title_list = []
for x in data2.title:
    title_list.append(" ".join(x))
title_list

['2 28 남양주 물류 창고 출 퇴근 셔틀 운행 실지 급 11 만 2',
 '2 28 냉장 풀타임 실지 급 11 만 2 천 당일 지급 하루 가능',
 '바로 출근 익일 지급 가능 저녁 출근 쿠팡 강서 1 캠프 장 단',
 '소분 포장 아르바이트',
 '블로그 포스팅 재택 알바 단기 바 부업 투 잡 가능 사무',
 '쿠팡 동탄 하루 가능 친구 와 함께 가능 초보 가능 주간 야',
 '쿠팡 동탄 하루 가능 친구 와 함께 가능 초보 가능 주간 야',
 '내일 급구 서울 장지 쿠팡 서울 단기 동반 주간 꿀 알바',
 '내일 급구 서울 장지 쿠팡 서울 단기 동반 주간 꿀 알바',
 '내일 급구 서울 장지 쿠팡 HUB 직영 C 팀 주간 단기 상차',
 '내일 급구 서울 장지 쿠팡 HUB 직영 C 팀 주간 단기 상차',
 '내일 급구 서울 장지 쿠팡 HUB 직영 C 팀 주간 단기 상차',
 '내일 급구 서울 장지 쿠팡 HUB 직영 C 팀 주간 단기 상차',
 '내일 급구 서울 장지 쿠팡 서울 단기 동반 주간 꿀 알바',
 '쿠팡 광주 주간 조 직영 C 팀 단기 꿀 바 동반 가능 물류',
 '쿠팡 광주 주간 조 직영 C 팀 단기 꿀 바 동반 가능 물류',
 '쿠팡 광주 주간 조 직영 C 팀 단기 꿀 바 동반 가능 물류',
 '쿠팡 광주 주간 조 직영 C 팀 단기 꿀 바 동반 가능 물류',
 '쿠팡 광주 HUB 주간 조 직영 C 팀 단기 상차 분류',
 '덕평 쿠팡 야간 일당 주휴 수당 피 킹 포장 단기 주말 바',
 '쿠팡 광주 HUB 주간 조 직영 C 팀 단기 상차 분류',
 '쿠팡 광주 HUB 주간 조 직영 C 팀 단기 상차 분류',
 '쿠팡 광주 HUB 주간 조 직영 C 팀 단기 상차 분류',
 '덕평 쿠팡 야간 일당 주휴 수당 피 킹 포장 단기 주말 바',
 '덕평 쿠팡 야간 일당 주휴 수당 피 킹 포장 단기 주말 바',
 '무스 100 출근 가능 합니다 친구 동반 가능 주휴 지급 초보',
 '주간 급구 초보 가능 단기 바 바 동반 동탄 쿠팡 E 팀',
 '마켓 컬

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

cv = CountVectorizer()
cv.fit(title_list)
print(cv.vocabulary_)
temp = cv.transform(title_list).toarray()

{'28': 10, '남양주': 38, '물류': 59, '창고': 139, '퇴근': 159, '셔틀': 83, '운행': 113, '실지': 95, '11': 3, '냉장': 40, '풀타임': 167, '당일': 48, '지급': 134, '하루': 169, '가능': 25, '바로': 60, '출근': 147, '익일': 118, '저녁': 126, '쿠팡': 154, '강서': 28, '캠프': 151, '소분': 84, '포장': 166, '아르바이트': 96, '블로그': 69, '포스팅': 165, '재택': 125, '알바': 98, '단기': 46, '부업': 66, '사무': 71, '동탄': 52, '친구': 150, '함께': 171, '초보': 143, '주간': 127, '내일': 39, '급구': 36, '서울': 78, '장지': 124, '동반': 51, 'hub': 19, '직영': 136, '상차': 75, '광주': 34, '분류': 68, '덕평': 50, '야간': 100, '일당': 121, '주휴': 130, '수당': 86, '주말': 128, '무스': 57, '100': 2, '합니다': 172, '마켓': 53, '컬리': 152, '실근': 94, '쉬움': 89, '112': 4, '000': 0, '29': 11, '부터': 67, '여의도역': 105, '양재역': 102, '집기': 138, '이동': 115, '모집': 56, 'nc': 22, '픽스': 168, '양일': 101, '급해요': 37, '오후': 110, '10': 1, '가구': 24, '설치': 81, '보조': 65, '패킹': 163, '업무': 103, '18': 8, '45': 12, '98': 18, '786': 16, '평일': 164, '2001': 9, '쉬운': 88, '72': 14, '160': 6, '단순': 47, '개월': 29, '호반': 175, '파크': 161, '건설': 30, '사옥': 73,

In [12]:
temp

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])